# Import dataset

In [19]:
import turicreate as tc
import matplotlib.pyplot as plt

In [20]:
images_dir = "../../data/"
train_data = tc.image_analysis.load_images(images_dir + "train", with_path=True)
val_data = tc.image_analysis.load_images(images_dir + "val", with_path=True)
test_data = tc.image_analysis.load_images(images_dir + "test", with_path=True)

Unsupported colorspace format. Currently, only RGB and Grayscale are supported.	 file: ../../data/train/ortolana/pizza-ortolana-a1742-16.jpg

In [21]:
import os

train_data["label"] = train_data["path"].apply(lambda path: os.path.basename(os.path.split(path)[0]))
val_data["label"] = val_data["path"].apply(lambda path: os.path.basename(os.path.split(path)[0]))
test_data["label"] = test_data["path"].apply(lambda path: os.path.basename(os.path.split(path)[0]))

# Training

Available models:
- squeezenet_v1.1
- resnet-50

In [22]:
# model = tc.image_classifier.create(train_data,
#                                    validation_set=val_data,
#                                    target="label",
#                                    model="squeezenet_v1.1",
#                                    verbose=True,
#                                    max_iterations=100,
#                                    l2_penalty=10.0,
#                                    l1_penalty=0.0,
#                                    convergence_threshold=1e-8
#                                   )

In [ ]:
# Extract and save features

from turicreate.toolkits import _pre_trained_models
from turicreate.toolkits import _image_feature_extractor

ptModel = _pre_trained_models.MODELS["squeezenet_v1.1"]()
feature_extractor = _image_feature_extractor.MXFeatureExtractor(ptModel)

train_features = feature_extractor.extract_features(train_data, "image", verbose=True)
extracted_train_features = tc.SFrame({"label": train_data["label"], "__image_features__": train_features})
extracted_train_features.save("train_features_tc.sframe")

val_features = feature_extractor.extract_features(val_data, "image", verbose=True)
extracted_val_features = tc.SFrame({"label": val_data["label"], "__image_features__": val_features})
extracted_val_features.save("val_features_tc.sframe")

Performing feature extraction on resized images...
Completed  64/958


In [ ]:
lr_model = tc.logistic_classifier.create(extracted_train_features,
                                         features=["__image_features__"],
                                         target="label",
                                         validation_set=extracted_val_features,
                                         max_iterations=200,
                                         seed=None,
                                         verbose=True,
                                         l2_penalty=10.0,
                                         l1_penalty=0.0,
                                         convergence_threshold=1e-8
                                        )

In [ ]:
from turicreate.toolkits.image_classifier import ImageClassifier

state = {
    'classifier': lr_model,
    'model': ptModel.name,
    'max_iterations': lr_model.max_iterations,
    'feature_extractor': feature_extractor,
    'input_image_shape': ptModel.input_image_shape,
    'target': lr_model.target,
    'feature': 'image',
    'num_features': 1,
    'num_classes': lr_model.num_classes,
    'classes': lr_model.classes,
    'num_examples': lr_model.num_examples,
    'training_time': lr_model.training_time,
    'training_loss': lr_model.training_loss
}

model = ImageClassifier(state)

# Testing

In [ ]:
metrics = model.evaluate(test_data)

In [ ]:
output = model.classify(test_data)

In [ ]:
predictions = model.predict(test_data, output_type="probability_vector")

In [ ]:
imgs_with_pred = test_data.add_columns(output)
imgs_filtered = imgs_with_pred[(imgs_with_pred["probability"] > 0.9) & (imgs_with_pred["label"] != imgs_with_pred["class"])]

In [ ]:
import numpy as np
import seaborn as sns

def compute_confusion_matrix(metrics, labels):
    num_labels = len(labels)
    label_to_index = {l:i for i,l in enumerate(labels)}
    
    conf = np.zeros((num_labels, num_labels), dtype=np.int)
    for row in metrics["confusion_matrix"]:
        true_label = label_to_index[row["target_label"]]
        pred_label = label_to_index[row["predicted_label"]]
        conf[true_label, pred_label] = row["count"]
        
    return conf

def plot_confusion_matrix(conf, labels, figsize=(8, 8)):
    fig = plt.figure(figsize=figsize)
    heatmap = sns.heatmap(conf, annot=True, fmt="d")
    heatmap.xaxis.set_ticklabels(labels, rotation=45, ha="right", fontsize=12)
    heatmap.yaxis.set_ticklabels(labels, rotation=0, ha="right", fontsize=12)
    plt.xlabel("Predicted label", fontsize=12)
    plt.ylabel("True label", fontsize=12)
    plt.show()

In [ ]:
labels = test_data["label"].unique().sort()
conf = compute_confusion_matrix(metrics, labels)
plot_confusion_matrix(conf, labels, figsize=(16, 16))

In [ ]:
for i, label in enumerate(labels):
    correct = conf[i, i]
    images_per_class = conf[i].sum()
    print("%10s %.1f%%" % (label, 100. * correct/images_per_class))

In [ ]:
class2index = {v:k for k,v in enumerate(test_data["label"].unique().sort())}
predicted_labels = np.argmax(predictions, axis=-1)
target_labels = list(map(lambda l: class2index[l], test_data["label"]))

In [ ]:
# Find for which images the predicted class is wrong
wrong_images = np.where(predicted_labels != target_labels)[0]

# For every prediction, find the largest probability value;
# this is the probability of the winning class for this image
probs_max = np.max(np.array(predictions), axis=-1)

# Sort the probabilities from the wrong images from low to high
idx = np.argsort(probs_max[wrong_images])

# Reverse the order (high to low), and keep the 5 highest ones
idx = idx[::-1][:5]

# Get the indices of the images with the worst predictions
worst_predictions = wrong_images[idx]

index2class = {k:v for k,v in enumerate(test_data["label"].unique().sort())}

for i in worst_predictions:
    print("%s was predicted as '%s' %.4f" % (test_data["path"][i],
                                             index2class[predicted_labels[i]],
                                             probs_max[i]))

# Convertion

In [ ]:
model.save("Pizza_tc.model")

In [ ]:
model.export_coreml("Pizza_tc.mlmodel")